In [1]:
import os

In [2]:
%pwd

'd:\\AI-Projects\\Chicken-diseases-classification\\research'

In [3]:
os.chdir('../')
%pwd

'd:\\AI-Projects\\Chicken-diseases-classification'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareCallbackConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [5]:
from ChickenDiseaseClassifier.constants import  *
from ChickenDiseaseClassifier.utils.common import  read_yaml,create_directories

In [6]:
class ConfigurationManager:
    def __init__(
        
        self,
        config_filepath= CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH
    ):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])


    def get_prepare_callback_config(self)->PrepareCallbackConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir= os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensor_root_log_dir)
        ])

        prepare_callback_config= PrepareCallbackConfig(
            root_dir= Path(config.root_dir),
            tensorboard_root_log_dir= Path(config.tensor_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )

        return prepare_callback_config

In [8]:
import os
import urllib.request as request
import zipfile
from ChickenDiseaseClassifier import logger
import tensorflow as tf 
import time


In [13]:
class PrepareCallback:
    def __init__(self, config: PrepareCallbackConfig):
        self.config = config
        
    @property
    def __create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}",
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    
    @property
    def __create_ckpt_callbacks(self):
        filepath = Path(self.config.checkpoint_model_filepath).with_suffix('.keras')
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=str(filepath),
            save_best_only=True,
    )

    
    def get_tb_ckpt_callbacks(self):
        return [self.__create_tb_callbacks, self.__create_ckpt_callbacks]
                                  



In [14]:
try:
    config = ConfigurationManager()
    prepare_callback_config = config.get_prepare_callback_config()
    prepare_callback= PrepareCallback(config=prepare_callback_config)
    callback_list= prepare_callback.get_tb_ckpt_callbacks()
    
except Exception as e:
    raise e

[2024-11-16 19:15:27,793:INFO:common:Successfully loaded YAML file: config\config.yaml]
[2024-11-16 19:15:27,803:INFO:common:Successfully loaded YAML file: params.yaml]
[2024-11-16 19:15:27,808:INFO:common:Directory created: artifacts]
[2024-11-16 19:15:27,812:INFO:common:Directory created: artifacts\prepare_callbacks\checkpoint_dir]
[2024-11-16 19:15:27,817:INFO:common:Directory created: artifacts\prepare_callbacks\tensor_log_dir]


TypeError: ModelCheckpoint.__init__() got an unexpected keyword argument 'save_format'